In [ ]:
import pandas as pd, import numpy as np
import tqdm as tdqm
from statsbombpy as sb
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression


# All public competitions
comps = sb.competitions()